# Cómo preparar un modelo de recomendador externo para evaluarlo con la librería Elliot

En este notebook se explica los pasos que se deben realizar para poder evaluar un sistema recomendador externo con la librería Elliot. Para ello, crearemos un sistema recomendador implementado con la librería [Surprise](https://surpriselib.com/). Para el sistema recomendador usaremos el dataset de [Movielens 1M](https://grouplens.org/datasets/movielens/1m/), cuyos ficheros se encuentran en la carpeta `data`.

## Paso 1 - Preparar el dataset

En este ejemplo solo usaremos el fichero que contiene los ratings. El contenido de este dataset lo dividiremos en:
- `train`: contiene el 80% de los datos y lo usaremos para entrenar el modelo.
- `test`: contiene el 20% de los datos y los usaremos para evaluar el modelo.

In [1]:
# Leemos el dataset

import pandas as pd

ratings_df = pd.read_csv('data/ml1m/ratings.dat', sep="::", header=None, engine='python')
ratings_df.columns = ["user_id", "movie_id", "rating", "timestamp"]
display(ratings_df.head())
print("# de ratings: ", ratings_df.shape[0])

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# de ratings:  1000209


In [2]:
# Creamos los datasets de train (80%) y test (20%)
import sklearn.model_selection as model_selection

train, test = model_selection.train_test_split(ratings_df, test_size=0.2, random_state=42)
print("# de ratings en train: ", train.shape[0])
print("# de ratings en test: ", test.shape[0])

# de ratings en train:  800167
# de ratings en test:  200042


## Paso 2: Entrenar el modelo de recomendación

En este ejemplo vamos a entrenar un modelo de recomendación basado en factorización de matrices no negativas (NMF). Para ello, usaremos la librería de surprise.

In [3]:
from surprise import Reader, Dataset, NMF

reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['user_id', 'movie_id', 'rating']], reader)

# Entrenamos el modelo
algo = NMF(n_factors=15, n_epochs=20, random_state=42)
algo.fit(train_data.build_full_trainset())

## Paso 3: Calculando las prediciones del modelo

A partir del dataset completo calculamos las predicciones para cada uno de los pares <usuario, item> que aparecen y los almacenamos en un columna nueva del dataframe.


In [4]:
ratings_df['prediction'] = ratings_df.apply(lambda x: algo.predict(x['user_id'], x['movie_id']).est, axis=1)
display(ratings_df.head())

,user_id,movie_id,rating,timestamp,prediction
0,1,1193,5,978300760,4.944651
1,1,661,3,978302109,3.856115
2,1,914,3,978301968,4.333493
3,1,3408,4,978300275,4.031539
4,1,2355,5,978824291,4.354696


## Paso 4: Creación de los ficheros para Elliot

El último paso es crear los ficheros `.tsv` que usará la librería elliot para hacer las evaluaciones.

In [6]:
# Creación del fichero de predicciones

# Seleccionamos las columnas que usará Elliot respetando el orden
predictions_to_save = ratings_df[['user_id', 'movie_id', 'prediction', 'timestamp']]

predictions_to_save.to_csv('data/elliot/predictions.tsv', sep="\t", header=None, index=False)

In [8]:
# Creación del fichero de entrenamiento
train.to_csv('data/elliot/train.tsv', sep="\t", header=None, index=False)

# Creación del fichero de test
test.to_csv('data/elliot/test.tsv', sep="\t", header=None, index=False)

## A continuación...

Una vez creados estos ficheros ya podemos hacer una evaluación con Elliot. Para ello, solo hay que ejecutar el script de Python `experiment.py` que aparece en este repositorio.